# Prerequsites

## Create Catalog and Scehema

In [0]:
%sql
DROP SCHEMA  IF EXISTS quickstart_schema CASCADE;
CREATE SCHEMA quickstart_schema;

## Create Delta Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS quickstart_schema.users(
  id INT,
  name STRING,
  dob DATE,
  email STRING,
  gender STRING,
  country STRING,
  region STRING,
  city STRING,
  asset INT,
  marital_status STRING
);
DESCRIBE EXTENDED quickstart_schema.users;

col_name,data_type,comment
id,int,null
name,string,null
dob,date,null
email,string,null
gender,string,null
country,string,null
region,string,null
city,string,null
asset,int,null
marital_status,string,null


In [0]:
%sql
SELECT * from quickstart_schema.users;

id,name,dob,email,gender,country,region,city,asset,marital_status


## Load Data into Delta Table

In [0]:
spark.read.csv(
    path="dbfs:/FileStore/synechron/user_dataset/users_001.csv",
    header=True,
    inferSchema=True,
).write.mode("overwrite").saveAsTable("quickstart_schema.users")

# Read delta table
spark.read.table("quickstart_schema.users").limit(4).display()

id,name,dob,email,gender,country,region,city,asset,marital_status
1,Heather Gibbs,2024-10-31,heathergibbs6243@gmail.com,Female,United States,Virginia,Virginia Beach,734388,Married
2,Herrod Petersen,2024-02-19,herrodpetersen@yahoomail.com,Male,United States,Arizona,Phoenix,113506,Single
3,Ocean Workman,2024-10-10,oceanworkman2328@ymail.com,Male,United States,Tennessee,Clarksville,139985,Married
4,Xaviera Maxwell,2025-03-09,xavieramaxwell@gmail.com,Transgender,United States,Ohio,Cleveland,511409,Married


# Schema Enforcement - Source with more columns

> Schema enforcement, also known as schema validation, is a safeguard in Delta Lake that ensures data quality by rejecting writes to a table that do not match the table's schema.

**Reference**

https://www.databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html



In [0]:
spark.read.csv(
    path="dbfs:/FileStore/synechron/user_dataset/users_006_new_column_education.csv",
    header=True,
    inferSchema=True,
).write.mode("append").saveAsTable("quickstart_schema.users")
spark.read.table("quickstart_schema.users").limit(4).display()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2692471806036561>:1
----> 1 spark.read.csv(
      2     path="dbfs:/FileStore/synechron/user_dataset/users_006_new_column_education.csv",
      3     header=True,
      4     inferSchema=True,
      5 ).write.mode("append").saveAsTable("quickstart_schema.users")
      6 spark.read.table("quickstart_schema.users").limit(4).display()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1520, in DataFrameWriter.saveAsTable(self, name, format, mode, partiti

# Schema Evolution -  Merge Schema

In [0]:
spark.read.csv(
    path="dbfs:/FileStore/synechron/user_dataset/users_006_new_column_education.csv",
    header=True,
    inferSchema=True,
).write.option("mergeSchema", "true").mode("append").saveAsTable(
    "quickstart_schema.users"
)

# Read Delta table
spark.read.table("quickstart_schema.users").limit(4).display()

id,name,dob,email,gender,country,region,city,asset,marital_status,education
1,Heather Gibbs,2024-10-31,heathergibbs6243@gmail.com,Female,United States,Virginia,Virginia Beach,734388,Married,null
2,Herrod Petersen,2024-02-19,herrodpetersen@yahoomail.com,Male,United States,Arizona,Phoenix,113506,Single,null
3,Ocean Workman,2024-10-10,oceanworkman2328@ymail.com,Male,United States,Tennessee,Clarksville,139985,Married,null
4,Xaviera Maxwell,2025-03-09,xavieramaxwell@gmail.com,Transgender,United States,Ohio,Cleveland,511409,Married,null


In [0]:
%sql
DESCRIBE FORMATTED quickstart_schema.users;

col_name,data_type,comment
id,int,null
name,string,null
dob,date,null
email,string,null
gender,string,null
country,string,null
region,string,null
city,string,null
asset,int,null
marital_status,string,null


In [0]:
from pyspark.sql.functions import col
spark.read.table("quickstart_schema.users").filter(col("id")<5).limit(4).display()

id,name,dob,email,gender,country,region,city,asset,marital_status,education
1,Heather Gibbs,2024-10-31,heathergibbs6243@gmail.com,Female,United States,Virginia,Virginia Beach,734388,Married,null
2,Herrod Petersen,2024-02-19,herrodpetersen@yahoomail.com,Male,United States,Arizona,Phoenix,113506,Single,null
3,Ocean Workman,2024-10-10,oceanworkman2328@ymail.com,Male,United States,Tennessee,Clarksville,139985,Married,null
4,Xaviera Maxwell,2025-03-09,xavieramaxwell@gmail.com,Transgender,United States,Ohio,Cleveland,511409,Married,null


# Source with less columns

In [0]:
spark.read.csv(
    path="dbfs:/FileStore/synechron/user_dataset/users_012_less_columns.csv",
    header=True,
    inferSchema=True,
).write.mode("append").saveAsTable("quickstart_schema.users")

spark.read.table("quickstart_schema.users").filter(col("id") > 5501).limit(4).display()

id,name,dob,email,gender,country,region,city,asset,marital_status,education
5502,Thaddeus Bradley,null,null,null,null,null,null,null,null,null
5503,Brandon Randall,null,null,null,null,null,null,null,null,null
5504,Duncan Valdez,null,null,null,null,null,null,null,null,null
5505,Azalia Montgomery,null,null,null,null,null,null,null,null,null


# Source with different column names

In [0]:

spark.read.csv(
    path="dbfs:/FileStore/synechron/user_dataset/users_013_less_columns_with_different_column_names.csv",
    header=True,
    inferSchema=True,
).write.mode("append").saveAsTable("quickstart_schema.users")

spark.read.table("quickstart_schema.users").filter(col("id") >= 6001).limit(4).display()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2692471806036570>:1
----> 1 spark.read.csv(
      2     path="dbfs:/FileStore/synechron/user_dataset/users_013_less_columns_with_different_column_names.csv",
      3     header=True,
      4     inferSchema=True,
      5 ).write.mode("append").saveAsTable("quickstart_schema.users")
      7 spark.read.table("quickstart_schema.users").filter(col("id") >= 6001).limit(4).display()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1520, in DataFrameWriter

# Source with different data types

**Note**
Cannot Merge Incompatible  Schema

In [0]:

spark.read.csv(
    path="dbfs:/FileStore/synechron/user_dataset/users_012_datatype_mismatch_dob.csv",
    header=True,
    inferSchema=True,
).write.mode("append").saveAsTable("quickstart_schema.users")

spark.read.table("quickstart_schema.users").filter(col("id") == 5501).display()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-13052135826420>:1
----> 1 spark.read.csv(
      2     path="dbfs:/FileStore/synechron/user_dataset/users_012_datatype_mismatch_dob.csv",
      3     header=True,
      4     inferSchema=True,
      5 ).write.mode("append").saveAsTable("quickstart_schema.users")
      7 spark.read.table("quickstart_schema.users").filter(col("id") == 5501).display()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1520, in DataFrameWriter.saveAsTable(self, name, forma